In [54]:
# !pip install missingno
# !pip install geopy

In [55]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
AirbnbBerlin_df = pd.read_csv('/content/drive/My Drive/Airbnb/Airbnb Berlin.csv', index_col=0)
# AirbnbBerlin_df = pd.read_csv('./content/Airbnb Berlin.csv')
# df_2019 = AirbnbBerlin_df[AirbnbBerlin_df['review_date'].astype(str).str[6:] == '19']

# df_2019 = pd.read_csv('/content/drive/My Drive/Airbnb/AirbnbBerlin_2019.csv', index_col=0)
# df_2019 = pd.read_csv('./content/AirbnbBerlin_2019.csv')

In [58]:
AirbnbBerlin_df.groupby(['Listing ID', 'Host ID']).size().sort_values(ascending=False)

,,0
Listing ID,Host ID,
292864,286494,545
517425,2547002,526
652366,286494,496
127708,632174,479
571751,2547002,459
...,...,...
20069673,142645164,1
20071123,92834289,1
20081217,21547435,1


In [59]:
AirbnbBerlin_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 456961 entries, 0 to 456960
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Review ID              452805 non-null  float64
 1   review_date            452805 non-null  object 
 2   Reviewer ID            452805 non-null  float64
 3   Reviewer Name          452805 non-null  object 
 4   Comments               452595 non-null  object 
 5   Listing ID             456961 non-null  int64  
 6   Listing URL            456961 non-null  object 
 7   Listing Name           456756 non-null  object 
 8   Host ID                456961 non-null  int64  
 9   Host URL               456961 non-null  object 
 10  Host Name              456913 non-null  object 
 11  Host Since             456913 non-null  object 
 12  Host Response Time     398194 non-null  object 
 13  Host Response Rate     398194 non-null  object 
 14  Is Superhost           456913 non-null  o

In [60]:
df = AirbnbBerlin_df.copy()

# Data Preparation

### Pre-cleaning

In [61]:
# Display shape of "AirbnbBerlin_df"
df.shape

(456961, 46)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 456961 entries, 0 to 456960
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Review ID              452805 non-null  float64
 1   review_date            452805 non-null  object 
 2   Reviewer ID            452805 non-null  float64
 3   Reviewer Name          452805 non-null  object 
 4   Comments               452595 non-null  object 
 5   Listing ID             456961 non-null  int64  
 6   Listing URL            456961 non-null  object 
 7   Listing Name           456756 non-null  object 
 8   Host ID                456961 non-null  int64  
 9   Host URL               456961 non-null  object 
 10  Host Name              456913 non-null  object 
 11  Host Since             456913 non-null  object 
 12  Host Response Time     398194 non-null  object 
 13  Host Response Rate     398194 non-null  object 
 14  Is Superhost           456913 non-null  o

In [63]:
# Display head(2) of remaining "df"
df.head(2)

,Review ID,review_date,Reviewer ID,Reviewer Name,Comments,Listing ID,Listing URL,Listing Name,Host ID,Host URL,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,City,Postal Code,Country Code,Country,Latitude,Longitude,Is Exact Location,Property Type,Room Type,Accomodates,Bathrooms,Bedrooms,Beds,Square Feet,Price,Guests Included,Min Nights,Reviews,First Review,Last Review,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,58660447.0,01-03-16,14876562.0,Dennis,"Very nice, clean appartement! Josh was really ...",10002699,https://www.airbnb.com/rooms/10002699,Trendy and peaceful KreuzkÃ¶lln,44696672,https://www.airbnb.com/users/show/44696672,Josh,09-20-15,NaN,NaN,f,NeukÃ¶lln,NeukÃ¶lln,Berlin,12047.0,DE,Germany,52.49234,13.42994,t,Apartment,Entire home/apt,4,1.0,2.0,2.0,NaN,60.0,1,1,1,01-03-16,01-03-16,100.0,8.0,10.0,6.0,10.0,10.0,10.0,f,f
1,NaN,NaN,NaN,NaN,NaN,10002922,https://www.airbnb.com/rooms/10002922,Sehr zentrale Wohnung in Berlin,45848709,https://www.airbnb.com/users/show/45848709,Belinda,10-05-15,NaN,NaN,f,Prenzlauer Berg,Pankow,Berlin,10405.0,DE,Germany,52.53292,13.41502,f,Apartment,Entire home/apt,2,1.0,1.0,1.0,NaN,52.0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f


On first sight, most features appear relatively informative and well-structured. comments stores reviewer's comments as a string, which needs to be reviewed later on.
At this point point, will move comments to a different dataset and store in sparated CSV file comments.csv

In [64]:
# df_comments = df[['Host ID', 'Listing ID', 'Review ID', 'Reviewer ID', 'Reviewer Name', 'Comments']].copy()
# df_comments.to_csv('/content/drive/My Drive/Airbnb/comments.csv')

Now we saved comment into different dataset, will drop it with other redundant columns

In [65]:
# df = df.drop(columns=['Review ID', 'Reviewer ID', 'Reviewer Name', 'Listing URL','Listing Name',
#                       'Host URL', 'Host Name', 'City', 'Country Code', 'Country','Comments'])

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 456961 entries, 0 to 456960
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Review ID              452805 non-null  float64
 1   review_date            452805 non-null  object 
 2   Reviewer ID            452805 non-null  float64
 3   Reviewer Name          452805 non-null  object 
 4   Comments               452595 non-null  object 
 5   Listing ID             456961 non-null  int64  
 6   Listing URL            456961 non-null  object 
 7   Listing Name           456756 non-null  object 
 8   Host ID                456961 non-null  int64  
 9   Host URL               456961 non-null  object 
 10  Host Name              456913 non-null  object 
 11  Host Since             456913 non-null  object 
 12  Host Response Time     398194 non-null  object 
 13  Host Response Rate     398194 non-null  object 
 14  Is Superhost           456913 non-null  o

As expected, various numerical features are currently stored as objects and need to be transformed (e.g. Price, Host Response Rate, First Review, ...). Additionally, there are actually quite a few columns with missing values.

In [67]:
df.groupby('Listing ID').size().sort_values(ascending=False)

,0
Listing ID,
292864,545
517425,526
652366,496
127708,479
571751,459
...,...
20069673,1
20071123,1
20081217,1


Notice, 'Listing ID' is repeated in mutiple rows, which means that the data need to be aggregated by 'Listing ID'.
Before aggregation we will Perform text cleaning, remove currency symbols & commas of some of clumns and clean and convert the numeric columns as was observed previously , which currently stored as an objects like 'Price' and 'Host Response Rate.

In [68]:
print(df.select_dtypes(include=['object']).columns)
# categorical_cols

Index(['review_date', 'Reviewer Name', 'Comments', 'Listing URL',
       'Listing Name', 'Host URL', 'Host Name', 'Host Since',
       'Host Response Time', 'Host Response Rate', 'Is Superhost',
       'neighbourhood', 'Neighborhood Group', 'City', 'Postal Code',
       'Country Code', 'Country', 'Is Exact Location', 'Property Type',
       'Room Type', 'Price', 'First Review', 'Last Review', 'Instant Bookable',
       'Business Travel Ready'],
      dtype='object')


As expected, various numerical features are currently stored as objects and need to be transformed (e.g. Price, Host Response Rate, First Review, ...). Additionally, there are actually quite a few columns with missing values.

In [69]:
# 1. Clean Text: Perform text cleaning, remove currency symbols & commas
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)
df['Host Response Rate'] = df['Host Response Rate'].replace('[\%,]', '', regex=True).astype(float)
df['Host Response Rate'] = df['Host Response Rate'].astype(float)

# Fix Postal Code incorrect values, remove '\n' and other irrelevant text
df['Postal Code'] = df['Postal Code'].astype(str).str[:5]

In [70]:
# Aggregate the dataset by 'Listing ID'
# For numerical columns, we'll compute the mean
# For categorical columns, we'll take the first value (assuming consistency)
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['number']).columns.difference(['Listing ID'])
aggregated_df = df.groupby('Listing ID').agg({**{col: 'mean' for col in numerical_cols},
                                              **{col: 'last' for col in categorical_cols}})

# aggregated_df = df.groupby('Listing ID').agg({**{col: 'mean' for col in numerical_cols},
#                                               **{col: lambda x: x.mode().iloc[0] if not x.mode().empty else None for col in categorical_cols}})

print(aggregated_df.shape)
df = aggregated_df.copy()

(23536, 45)


### Insepction

In [71]:
# Describe data (summary)
df.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
Accomodates,23536.0,2.670000e+00,1.570000e+00,1.00,2.000000e+00,2.000000e+00,3.000000e+00,1.600000e+01
Accuracy Rating,18888.0,9.680000e+00,7.400000e-01,2.00,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01
Bathrooms,23507.0,1.100000e+00,3.500000e-01,0.00,1.000000e+00,1.000000e+00,1.000000e+00,8.500000e+00
Bedrooms,23516.0,1.160000e+00,6.600000e-01,0.00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+01
Beds,23501.0,1.640000e+00,1.230000e+00,0.00,1.000000e+00,1.000000e+00,2.000000e+00,2.200000e+01
Checkin Rating,18870.0,9.730000e+00,7.000000e-01,2.00,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01
Cleanliness Rating,18892.0,9.330000e+00,1.030000e+00,2.00,9.000000e+00,1.000000e+01,1.000000e+01,1.000000e+01
Communication Rating,18886.0,9.750000e+00,6.800000e-01,2.00,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01
Guests Included,23536.0,1.350000e+00,8.600000e-01,1.00,1.000000e+00,1.000000e+00,1.000000e+00,1.600000e+01
Host ID,23536.0,6.172301e+07,6.782382e+07,1581.00,9.819079e+06,3.424883e+07,9.412761e+07,2.616142e+08


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23536 entries, 2695 to 34682315
Data columns (total 45 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Accomodates            23536 non-null  float64
 1   Accuracy Rating        18888 non-null  float64
 2   Bathrooms              23507 non-null  float64
 3   Bedrooms               23516 non-null  float64
 4   Beds                   23501 non-null  float64
 5   Checkin Rating         18870 non-null  float64
 6   Cleanliness Rating     18892 non-null  float64
 7   Communication Rating   18886 non-null  float64
 8   Guests Included        23536 non-null  float64
 9   Host ID                23536 non-null  float64
 10  Host Response Rate     13046 non-null  float64
 11  Latitude               23536 non-null  float64
 12  Location Rating        18871 non-null  float64
 13  Longitude              23536 non-null  float64
 14  Min Nights             23536 non-null  float64
 15  O

In [73]:
# Show maximum/minimum value for each numerical column
num_features = df.columns[df.dtypes!=object].difference(['Review ID','Reviewer ID','Listing ID','Host ID','Latitude','Longitude'])
df[num_features].describe().loc[['min','max']].T

,min,max
Accomodates,1.0,16.0
Accuracy Rating,2.0,10.0
Bathrooms,0.0,8.5
Bedrooms,0.0,10.0
Beds,0.0,22.0
Checkin Rating,2.0,10.0
Cleanliness Rating,2.0,10.0
Communication Rating,2.0,10.0
Guests Included,1.0,16.0
Host Response Rate,0.0,100.0


Several rows with unusually high values can be identified and may in some cases be dropped at a certain threshold during data handling and before aggrgation step. Some particular features include:

In [74]:
df[['Beds','Min Nights','Reviews']].describe().loc[['max']].T

,max
Beds,22.0
Min Nights,1000.0
Reviews,545.0


In [75]:
# df = df[df['Beds'] <= 20].groupby('Listing ID').first()

In [76]:
df.nunique().sort_values(ascending=False)

,0
Listing URL,23536
Listing Name,22837
Host ID,19772
Host URL,19772
Review ID,19380
Reviewer ID,19372
Comments,18237
Longitude,14851
Latitude,11821
Reviewer Name,6943


Three main insights from unique values:

- Some columns have only 1 value and can be dropped like 'Business Travel Ready'
- Some other columns have 2 values and appear to be true/false (i.e. can be recoded as 1/0)
- Certain columns have a high number of unique values, which can probably be clustered into a few relevant ones (e.g. neighbourhood, property_type)

List missing values (pre-cleaning)

In [77]:
# List missing values (pre-cleaning)
null_cols = df.columns[df.isnull().any(axis=0)]
df[null_cols].isna().sum().sort_values(ascending=False)

,0
Square Feet,23111
Host Response Rate,10490
Host Response Time,10490
Value Rating,4668
Checkin Rating,4666
Location Rating,4665
Communication Rating,4650
Accuracy Rating,4648
Cleanliness Rating,4644
Overall Rating,4622


Various features have a lot of missing values. In particular, there is an observable cut where many features have more than 4500 missing values and the rest has less than 1.000. The former - except for review_scores - shall be removed, the latter imputed.

Conclusions (selection)

- 'Square Feet' is unfortunately not available for more than half of the dataset and consequently the columns are removed
- 'Host Response Rate' and 'Host Response Time' are difficult to replace if they do not exist and if the host is not repated

### Aggregate dataset by Listing ID

Clean text From pancutations or undesired characters

In [78]:
# 1. Clean Text: Perform text cleaning, remove currency symbols & commas
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)
df['Host Response Rate'] = df['Host Response Rate'].replace('[\%,]', '', regex=True).astype(float)
df['Host Response Rate'] = df['Host Response Rate'].astype(float)

# Fix Postal Code incorrect values, remove '\n' and other irrelevant text
df['Postal Code'] = df['Postal Code'].astype(str).str[:5]

Aggregate the dataset by 'Listing ID'
- For numerical columns, we'll compute the mean
- For categorical columns, we'll take the first/or last value (assuming consistency)

In [79]:
# Aggregate the dataset by 'Listing ID'
# For numerical columns, we'll compute the mean
# For categorical columns, we'll take the first value (assuming consistency)
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['number']).columns.difference(['Listing ID'])
aggregated_df = df.groupby('Listing ID').agg({**{col: 'mean' for col in numerical_cols},
                                              **{col: 'last' for col in categorical_cols}})

# aggregated_df = df.groupby('Listing ID').agg({**{col: 'mean' for col in numerical_cols},
#                                               **{col: lambda x: x.mode().iloc[0] if not x.mode().empty else None for col in categorical_cols}})

print(aggregated_df.shape)
df = aggregated_df.copy()

(23536, 45)


### Features Selection (pre-cleaning)

1. If a categorical column is not relevant to the analysis, we can remove it.
2. Listing URL, Listing Name, Host URL, Host Name: These are mostly unique to each listing, so not useful for category reduction
3. columns with 50% missing values will be dropped like 'Square Feet'

#### List missing values

In [80]:
# List missing values (pre-cleaning)
null_cols = df.columns[df.isnull().any(axis=0)]
df[null_cols].isna().sum().sort_values(ascending=False)

,0
Square Feet,23111
Host Response Rate,10490
Host Response Time,10490
Value Rating,4668
Checkin Rating,4666
Location Rating,4665
Communication Rating,4650
Accuracy Rating,4648
Cleanliness Rating,4644
Overall Rating,4622


Various features have a lot of missing values. In particular, there is an observable cut where many features have more than 4.500 missing values and the rest has less than 1.000. The former - except for review_scores - shall be removed, the latter imputed.

#### List unique entries

In [81]:
# List unique entries per column
aggregated_df.nunique().sort_values(ascending=False)

,0
Listing URL,23536
Listing Name,22837
Host ID,19772
Host URL,19772
Review ID,19380
Reviewer ID,19372
Comments,18237
Longitude,14851
Latitude,11821
Reviewer Name,6943


Three main insights from unique values:

- Some columns have only 1 value and can be dropped
- Some other columns have 2 values and appear to be true/false (i.e. can be recoded as 1/0)
- Certain columns have a high number of unique values, which can probably be clustered into a few relevant ones (e.g. cancellation_policy, property_type)

#### Conclusions (selection) - dropping redundant columns

1. If a categorical column is not relevant to the analysis, we can remove it.
2. Listing URL, Listing Name, Host URL, Host Name: These are mostly unique to each listing, so not useful for category reduction
3. columns with more than 50% missing values will be dropped like 'Square Feet'
4. 'Comment' columns can be dropped as it was stored in separated dataset
5. 'Business Travel Ready' has one value and can be dropped

In [82]:
# drop the columns that is not helpful for prediction
df = df.drop(columns=['Review ID', 'Reviewer ID', 'Reviewer Name', 'Listing URL','Listing Name',
                      'Host ID', 'Host URL', 'Host Name', 'City', 'Country Code', 'Country',
                      'Square Feet', 'Business Travel Ready'])

### Reduce Large Categories

1. Group Rare Categories: If a categorical column has many unique values, we can group infrequent categories into an "Other" category like 'Reviewer Name'.
2. Merge Similar Categories: If there are similar categories (e.g., different spellings or formats of the same category), we can merge them.
3. Binning: For numerical categories (like "Overall Rating" or "Accommodates"), we can create bins to reduce the number of unique values.

In [83]:
# 2. Grouping neighbourhoods into Neighborhood Groups
if 'Neighborhood Group' in df.columns:
  neighbourhood_mapping = df.groupby('neighbourhood')['Neighborhood Group'].first()
  df['Neighbourhood Grouped'] = df['neighbourhood'].map(neighbourhood_mapping)

# 3. Reducing Property Types
property_mapping = {
    "Villa": "Vacation Rental",
    "Cottage": "Vacation Rental",
    "Bungalow": "Vacation Rental",
    "Cabin": "Vacation Rental",
    "Tiny house": "Vacation Rental",
    "Earth house": "Vacation Rental",
    "Treehouse": "Vacation Rental",
    "Hut": "Vacation Rental",
    "Barn": "Vacation Rental",
    "Houseboat": "Boats & Houseboats",
    "Boat": "Boats & Houseboats",
    "Camper/RV": "Mobile/Alternative Lodging",
    "Cave": "Mobile/Alternative Lodging",
    "Pension (South Korea)": "Mobile/Alternative Lodging",
    "Casa particular (Cuba)": "Mobile/Alternative Lodging",
}

# Apply mapping and assign 'Other' to rare categories
top_property_types = [
    "Apartment", "Loft", "House", "Townhouse", "Condominium", "Serviced apartment",
    "Hotel", "Hostel", "Guesthouse", "Bed and breakfast", "Boutique hotel"
]

df['Property Type Reduced'] = df['Property Type'].apply(
    lambda x: property_mapping.get(x, x) if x in top_property_types or x in property_mapping else "Other"
)

# 4. Binning Postal Codes (first two digits represent broad area)
df['Postal Code Reduced'] = df['Postal Code'].astype(str).str[:2]  # Use only first 2 digits

### Transform/Manipulate data

In [84]:
# Extracting years from date columns
df['Host Since'] = pd.to_datetime(df['Host Since'])
df['Host Since Year'] = df['Host Since'].dt.year

df['First Review'] = pd.to_datetime(df['First Review'])
df['First Review Year'] = df['First Review'].dt.year
df['First Review Month'] = df['First Review'].dt.month
df['First Review Day'] = df['First Review'].dt.day

df['Last Review'] = pd.to_datetime(df['Last Review'])
df['Last Review Year'] = df['Last Review'].dt.year
df['Last Review Month'] = df['Last Review'].dt.month
df['Last Review Day'] = df['Last Review'].dt.day

df['review_date'] = pd.to_datetime(df['review_date'])
df['Review Date Year'] = df['review_date'].dt.year
df['Review Date Month'] = df['review_date'].dt.month
df['Review Date Day'] = df['review_date'].dt.day

# 5. transform true/false into bool
df['Instant Bookable'] = df['Instant Bookable'].replace({'t': True, 'f': False})
df['Is Superhost'] = df['Is Superhost'].replace({'t': True, 'f': False})
df['Is Exact Location'] = df['Is Exact Location'].replace({'t': True, 'f': False})

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23536 entries, 2695 to 34682315
Data columns (total 45 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Accomodates            23536 non-null  float64       
 1   Accuracy Rating        18888 non-null  float64       
 2   Bathrooms              23507 non-null  float64       
 3   Bedrooms               23516 non-null  float64       
 4   Beds                   23501 non-null  float64       
 5   Checkin Rating         18870 non-null  float64       
 6   Cleanliness Rating     18892 non-null  float64       
 7   Communication Rating   18886 non-null  float64       
 8   Guests Included        23536 non-null  float64       
 9   Host Response Rate     13046 non-null  float64       
 10  Latitude               23536 non-null  float64       
 11  Location Rating        18871 non-null  float64       
 12  Longitude              23536 non-null  float64       
 13  

In [86]:
df.head(2)

,Accomodates,Accuracy Rating,Bathrooms,Bedrooms,Beds,Checkin Rating,Cleanliness Rating,Communication Rating,Guests Included,Host Response Rate,Latitude,Location Rating,Longitude,Min Nights,Overall Rating,Price,Reviews,Value Rating,review_date,Comments,Host Since,Host Response Time,Is Superhost,neighbourhood,Neighborhood Group,Postal Code,Is Exact Location,Property Type,Room Type,First Review,Last Review,Instant Bookable,Neighbourhood Grouped,Property Type Reduced,Postal Code Reduced,Host Since Year,First Review Year,First Review Month,First Review Day,Last Review Year,Last Review Month,Last Review Day,Review Date Year,Review Date Month,Review Date Day
Listing ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2695,2.0,10.0,1.0,1.0,1.0,10.0,10.0,10.0,1.0,50.0,52.54851,9.0,13.40455,2.0,100.0,17.0,7.0,10.0,2019-04-21,Michael is kind host and our stay was very ple...,2008-09-16,within a day,False,Prenzlauer Berg,Pankow,10437,True,Apartment,Private room,2018-07-04,2019-04-21,False,Pankow,Apartment,10,2008.0,2018.0,7.0,4.0,2019.0,4.0,21.0,2019.0,4.0,21.0
3176,4.0,9.0,1.0,1.0,2.0,9.0,9.0,9.0,2.0,50.0,52.53500,10.0,13.41758,62.0,92.0,90.0,144.0,9.0,2018-10-29,Good,2008-10-19,within a day,False,Prenzlauer Berg,Pankow,10405,True,Apartment,Entire home/apt,2009-06-20,2018-10-29,False,Pankow,Apartment,10,2008.0,2009.0,6.0,20.0,2018.0,10.0,29.0,2018.0,10.0,29.0


### Pre-EDA (Exploratory Data Analysis)

In [87]:
df.reset_index(inplace=True)

In [88]:
df_EDA = df.copy()
df_EDA.to_csv('/content/drive/My Drive/Airbnb/df_EDA.csv')

In [89]:
import pickle
with open('/content/drive/My Drive/Airbnb/df_EDA.pkl', 'wb') as f:
    pickle.dump(df_EDA, f)